In [9]:
import os
import pandas as pd
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
# tpu 사용을 위한 준비
tpu_name = 'grpc://'+os.environ['COLAB_TPU_ADDR']

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=tpu_name)

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

strategy = tf.distribute.TPUStrategy(resolver)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_dir = '/content/drive/MyDrive/Colab Notebooks/git/data/4d-block'
train_df = pd.read_csv(os.path.join(data_dir,'train.csv'))

In [5]:
train_df.head()

,id,img_path,A,B,C,D,E,F,G,H,I,J
0,TRAIN_00000,./train/TRAIN_00000.jpg,1,0,0,0,0,0,0,0,0,0
1,TRAIN_00001,./train/TRAIN_00001.jpg,1,0,0,0,0,0,0,0,0,0
2,TRAIN_00002,./train/TRAIN_00002.jpg,1,0,0,0,0,0,0,0,0,0
3,TRAIN_00003,./train/TRAIN_00003.jpg,1,0,0,0,0,0,0,0,0,0
4,TRAIN_00004,./train/TRAIN_00004.jpg,1,0,0,0,0,0,0,0,0,0


In [7]:
train_df.isnull().sum()

id          0
img_path    0
A           0
B           0
C           0
D           0
E           0
F           0
G           0
H           0
I           0
J           0
dtype: int64

In [10]:
train, val = train_test_split(train_df, test_size=0.35, random_state=2023)

In [12]:
train.shape, val.shape

((21446, 12), (11548, 12))